In [6]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.utils.class_weight import compute_class_weight

In [7]:
%%capture
!pip install xgboost
!pip install catboost
!pip install optuna

In [8]:
pd.set_option("display.max_columns", None)

# TESS (TOI)

In [9]:
df = pd.read_csv('./data_filtered/TOI_2025.10.03_08.53.59.csv')
print(df.shape)
df = df[df["tfopwg_disp"].isin(["PC", "FP", "CP"])]
df["tfopwg_disp"] = df["tfopwg_disp"].replace({
    "PC": "CANDIDATE",
    "FP": "FALSE POSITIVE",
    "CP": "CONFIRMED",
})
df.head()

(7703, 66)


,loc_rowid,toi,tid,tfopwg_disp,rastr,ra,decstr,dec,st_pmra,st_pmraerr1,st_pmraerr2,st_pmralim,st_pmdec,st_pmdecerr1,st_pmdecerr2,st_pmdeclim,pl_tranmid,pl_tranmiderr1,pl_tranmiderr2,pl_tranmidlim,pl_orbper,pl_orbpererr1,pl_orbpererr2,pl_orbperlim,pl_trandurh,pl_trandurherr1,pl_trandurherr2,pl_trandurhlim,pl_trandep,pl_trandeperr1,pl_trandeperr2,pl_trandeplim,pl_rade,pl_radeerr1,pl_radeerr2,pl_radelim,pl_insol,pl_insolerr1,pl_insolerr2,pl_insollim,pl_eqt,pl_eqterr1,pl_eqterr2,pl_eqtlim,st_tmag,st_tmagerr1,st_tmagerr2,st_tmaglim,st_dist,st_disterr1,st_disterr2,st_distlim,st_teff,st_tefferr1,st_tefferr2,st_tefflim,st_logg,st_loggerr1,st_loggerr2,st_logglim,st_rad,st_raderr1,st_raderr2,st_radlim,toi_created,rowupdate
0,1,1000.01,50365310,FALSE POSITIVE,07h29m25.85s,112.357708,-12d41m45.46s,-12.695960,-5.964,0.085,-0.085,0.0,-0.076,0.072,-0.072,0.0,2.459230e+06,0.001657,-0.001657,0,2.171348,0.000264,-0.000264,0,2.01722,0.319588,-0.319588,0,656.886099,37.778210,-37.778210,0,5.818163,1.910546,-1.910546,0,22601.948581,NaN,NaN,NaN,3127.204052,NaN,NaN,NaN,9.604000,0.013,-0.013,0,485.735,11.9515,-11.9515,0,10249.0,264.7,-264.7,0,4.19,0.07,-0.07,0,2.16986,0.072573,-0.072573,0,2019-07-24 15:58:33,2024-09-09 10:08:01
1,2,1001.01,88863718,CANDIDATE,08h10m19.31s,122.580465,-05d30m49.87s,-5.513852,-4.956,0.102,-0.102,0.0,-15.555,0.072,-0.072,0.0,2.459988e+06,0.001916,-0.001916,0,1.931646,0.000005,-0.000005,0,3.16600,0.647000,-0.647000,0,1286.000000,1186.490000,-1186.490000,0,11.215400,2.624200,-2.624200,0,44464.500000,NaN,NaN,NaN,4045.000000,NaN,NaN,NaN,9.423440,0.006,-0.006,0,295.862,5.9100,-5.9100,0,7070.0,126.4,-126.4,0,4.03,0.09,-0.09,0,2.01000,0.090000,-0.090000,0,2019-07-24 15:58:33,2023-04-03 14:31:04
2,3,1002.01,124709665,FALSE POSITIVE,06h58m54.47s,104.726966,-10d34m49.64s,-10.580455,-1.462,0.206,-0.206,0.0,-2.249,0.206,-0.206,0.0,2.459225e+06,0.000625,-0.000625,0,1.867557,0.000003,-0.000003,0,1.40800,0.184000,-0.184000,0,1500.000000,1.758400,-1.758400,0,23.752900,NaN,NaN,0,2860.610000,NaN,NaN,NaN,2037.000000,NaN,NaN,NaN,9.299501,0.058,-0.058,0,943.109,106.3330,-106.3330,0,8924.0,124.0,-124.0,0,NaN,NaN,NaN,0,5.73000,NaN,NaN,0,2019-07-24 15:58:33,2022-07-11 16:02:02
3,4,1003.01,106997505,FALSE POSITIVE,07h22m14.39s,110.559945,-25d12m25.26s,-25.207017,-0.939,0.041,-0.041,0.0,1.640,0.055,-0.055,0.0,2.458493e+06,0.005350,-0.005350,0,2.743230,0.001080,-0.001080,0,3.16700,0.642000,-0.642000,0,383.410000,0.781988,-0.781988,0,NaN,NaN,NaN,0,1177.360000,NaN,NaN,NaN,1631.000000,NaN,NaN,NaN,9.300300,0.037,-0.037,0,7728.170,1899.5700,-1899.5700,0,5388.5,567.0,-567.0,0,4.15,1.64,-1.64,0,NaN,NaN,NaN,0,2019-07-24 15:58:33,2022-02-23 10:10:02
4,5,1004.01,238597883,FALSE POSITIVE,08h08m42.77s,122.178195,-48d48m10.12s,-48.802811,-4.496,0.069,-0.069,0.0,9.347,0.062,-0.062,0.0,2.459987e+06,0.003748,-0.003748,0,3.573014,0.000013,-0.000013,0,3.37000,1.029000,-1.029000,0,755.000000,1306.550000,-1306.550000,0,11.311300,3.247140,-3.247140,0,54679.300000,NaN,NaN,NaN,4260.000000,NaN,NaN,NaN,9.135500,0.006,-0.006,0,356.437,4.6175,-4.6175,0,9219.0,171.1,-171.1,0,4.14,0.07,-0.07,0,2.15000,0.060000,-0.060000,0,2019-07-24 15:58:33,2024-09-09 10:08:01


In [10]:
df2 = df[0:100]
df2.to_csv('./tess.csv',index=False)

In [11]:
df['tfopwg_disp'].value_counts()

tfopwg_disp
CANDIDATE         4679
FALSE POSITIVE    1197
CONFIRMED          684
Name: count, dtype: int64

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6560 entries, 0 to 7702
Data columns (total 66 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   loc_rowid        6560 non-null   int64  
 1   toi              6560 non-null   float64
 2   tid              6560 non-null   int64  
 3   tfopwg_disp      6560 non-null   object 
 4   rastr            6560 non-null   object 
 5   ra               6560 non-null   float64
 6   decstr           6560 non-null   object 
 7   dec              6560 non-null   float64
 8   st_pmra          6446 non-null   float64
 9   st_pmraerr1      6446 non-null   float64
 10  st_pmraerr2      6446 non-null   float64
 11  st_pmralim       6446 non-null   float64
 12  st_pmdec         6446 non-null   float64
 13  st_pmdecerr1     6446 non-null   float64
 14  st_pmdecerr2     6446 non-null   float64
 15  st_pmdeclim      6446 non-null   float64
 16  pl_tranmid       6560 non-null   float64
 17  pl_tranmiderr1   65

In [13]:
# df.drop(['loc_rowid','tid','toi','rastr','decstr','pl_insolerr1','pl_insolerr2',
#          'pl_insollim','pl_eqterr1','pl_eqterr2','pl_eqtlim','toi_created',
#          'rowupdate',
# "st_logglim",
# "st_pmralim",
# "st_tefflim",
# "pl_radelim",
# "pl_trandeplim",
# "st_pmdeclim",
# "st_distlim",
# "pl_tranmidlim",
# "pl_orbperlim",
# "pl_trandurhlim",
# "st_tmaglim",
# "st_tmagerr2",
# "st_radlim"
# ],axis=1,inplace=True)

df.drop(['loc_rowid','tid','toi','rastr','decstr','pl_insolerr1','pl_insolerr2',
         'pl_insollim','pl_eqterr1','pl_eqterr2','pl_eqtlim','toi_created',
         'rowupdate'
],axis=1,inplace=True)



In [14]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

# Aplicar no label
df["label"] = encoder.fit_transform(df["tfopwg_disp"])
encoder = LabelEncoder()
df.drop(['tfopwg_disp'],axis=1,inplace=True)
df.head()

,ra,dec,st_pmra,st_pmraerr1,st_pmraerr2,st_pmralim,st_pmdec,st_pmdecerr1,st_pmdecerr2,st_pmdeclim,pl_tranmid,pl_tranmiderr1,pl_tranmiderr2,pl_tranmidlim,pl_orbper,pl_orbpererr1,pl_orbpererr2,pl_orbperlim,pl_trandurh,pl_trandurherr1,pl_trandurherr2,pl_trandurhlim,pl_trandep,pl_trandeperr1,pl_trandeperr2,pl_trandeplim,pl_rade,pl_radeerr1,pl_radeerr2,pl_radelim,pl_insol,pl_eqt,st_tmag,st_tmagerr1,st_tmagerr2,st_tmaglim,st_dist,st_disterr1,st_disterr2,st_distlim,st_teff,st_tefferr1,st_tefferr2,st_tefflim,st_logg,st_loggerr1,st_loggerr2,st_logglim,st_rad,st_raderr1,st_raderr2,st_radlim,label
0,112.357708,-12.695960,-5.964,0.085,-0.085,0.0,-0.076,0.072,-0.072,0.0,2.459230e+06,0.001657,-0.001657,0,2.171348,0.000264,-0.000264,0,2.01722,0.319588,-0.319588,0,656.886099,37.778210,-37.778210,0,5.818163,1.910546,-1.910546,0,22601.948581,3127.204052,9.604000,0.013,-0.013,0,485.735,11.9515,-11.9515,0,10249.0,264.7,-264.7,0,4.19,0.07,-0.07,0,2.16986,0.072573,-0.072573,0,2
1,122.580465,-5.513852,-4.956,0.102,-0.102,0.0,-15.555,0.072,-0.072,0.0,2.459988e+06,0.001916,-0.001916,0,1.931646,0.000005,-0.000005,0,3.16600,0.647000,-0.647000,0,1286.000000,1186.490000,-1186.490000,0,11.215400,2.624200,-2.624200,0,44464.500000,4045.000000,9.423440,0.006,-0.006,0,295.862,5.9100,-5.9100,0,7070.0,126.4,-126.4,0,4.03,0.09,-0.09,0,2.01000,0.090000,-0.090000,0,0
2,104.726966,-10.580455,-1.462,0.206,-0.206,0.0,-2.249,0.206,-0.206,0.0,2.459225e+06,0.000625,-0.000625,0,1.867557,0.000003,-0.000003,0,1.40800,0.184000,-0.184000,0,1500.000000,1.758400,-1.758400,0,23.752900,NaN,NaN,0,2860.610000,2037.000000,9.299501,0.058,-0.058,0,943.109,106.3330,-106.3330,0,8924.0,124.0,-124.0,0,NaN,NaN,NaN,0,5.73000,NaN,NaN,0,2
3,110.559945,-25.207017,-0.939,0.041,-0.041,0.0,1.640,0.055,-0.055,0.0,2.458493e+06,0.005350,-0.005350,0,2.743230,0.001080,-0.001080,0,3.16700,0.642000,-0.642000,0,383.410000,0.781988,-0.781988,0,NaN,NaN,NaN,0,1177.360000,1631.000000,9.300300,0.037,-0.037,0,7728.170,1899.5700,-1899.5700,0,5388.5,567.0,-567.0,0,4.15,1.64,-1.64,0,NaN,NaN,NaN,0,2
4,122.178195,-48.802811,-4.496,0.069,-0.069,0.0,9.347,0.062,-0.062,0.0,2.459987e+06,0.003748,-0.003748,0,3.573014,0.000013,-0.000013,0,3.37000,1.029000,-1.029000,0,755.000000,1306.550000,-1306.550000,0,11.311300,3.247140,-3.247140,0,54679.300000,4260.000000,9.135500,0.006,-0.006,0,356.437,4.6175,-4.6175,0,9219.0,171.1,-171.1,0,4.14,0.07,-0.07,0,2.15000,0.060000,-0.060000,0,2


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6560 entries, 0 to 7702
Data columns (total 53 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ra               6560 non-null   float64
 1   dec              6560 non-null   float64
 2   st_pmra          6446 non-null   float64
 3   st_pmraerr1      6446 non-null   float64
 4   st_pmraerr2      6446 non-null   float64
 5   st_pmralim       6446 non-null   float64
 6   st_pmdec         6446 non-null   float64
 7   st_pmdecerr1     6446 non-null   float64
 8   st_pmdecerr2     6446 non-null   float64
 9   st_pmdeclim      6446 non-null   float64
 10  pl_tranmid       6560 non-null   float64
 11  pl_tranmiderr1   6550 non-null   float64
 12  pl_tranmiderr2   6550 non-null   float64
 13  pl_tranmidlim    6560 non-null   int64  
 14  pl_orbper        6471 non-null   float64
 15  pl_orbpererr1    6451 non-null   float64
 16  pl_orbpererr2    6451 non-null   float64
 17  pl_orbperlim     65

In [16]:
df = df.fillna(df.mean())
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6560 entries, 0 to 7702
Data columns (total 53 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ra               6560 non-null   float64
 1   dec              6560 non-null   float64
 2   st_pmra          6560 non-null   float64
 3   st_pmraerr1      6560 non-null   float64
 4   st_pmraerr2      6560 non-null   float64
 5   st_pmralim       6560 non-null   float64
 6   st_pmdec         6560 non-null   float64
 7   st_pmdecerr1     6560 non-null   float64
 8   st_pmdecerr2     6560 non-null   float64
 9   st_pmdeclim      6560 non-null   float64
 10  pl_tranmid       6560 non-null   float64
 11  pl_tranmiderr1   6560 non-null   float64
 12  pl_tranmiderr2   6560 non-null   float64
 13  pl_tranmidlim    6560 non-null   int64  
 14  pl_orbper        6560 non-null   float64
 15  pl_orbpererr1    6560 non-null   float64
 16  pl_orbpererr2    6560 non-null   float64
 17  pl_orbperlim     65

In [17]:
df['label'].value_counts()

label
0    4679
2    1197
1     684
Name: count, dtype: int64

## Treinamento e Teste

In [18]:
df_final = df[[
    "st_dist",
    "st_tmag",
    "pl_eqt",
    "pl_insol",
    "st_disterr2",
    "st_disterr1",
    "pl_rade",
    "pl_tranmid",
    "pl_tranmiderr2",
    "st_raderr2", "label"
]
]

In [19]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6560 entries, 0 to 7702
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   st_dist         6560 non-null   float64
 1   st_tmag         6560 non-null   float64
 2   pl_eqt          6560 non-null   float64
 3   pl_insol        6560 non-null   float64
 4   st_disterr2     6560 non-null   float64
 5   st_disterr1     6560 non-null   float64
 6   pl_rade         6560 non-null   float64
 7   pl_tranmid      6560 non-null   float64
 8   pl_tranmiderr2  6560 non-null   float64
 9   st_raderr2      6560 non-null   float64
 10  label           6560 non-null   int64  
dtypes: float64(10), int64(1)
memory usage: 615.0 KB


In [20]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score
import xgboost as xgb
import joblib
from time import time
from sklearn.metrics import accuracy_score, f1_score, classification_report


start = time()

# ==============================
# SPLITS E MODELO BASE
# ==============================

# Features e label
X = df_final.drop(['label'], axis=1)
y = df_final['label']

# Primeiro split: 70% treino, 30% restante
X_train_full, X_temp, y_train_full, y_temp = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=42
)

# Segundo split: 20% teste, 10% blind validation
X_test, X_blind, y_test, y_blind = train_test_split(
    X_temp, y_temp, test_size=0.3333, stratify=y_temp, random_state=42
)

print(f"Train shape: {X_train_full.shape}")
print(f"Test shape: {X_test.shape}")
print(f"Blind validation shape: {X_blind.shape}")

# ==============================
# CROSS-VALIDATION
# ==============================

n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
fold_results = []

for fold, (train_idx, val_idx) in enumerate(skf.split(X_train_full, y_train_full), 1):
    print(f"\nFold {fold}")
    X_train, X_val = X_train_full.iloc[train_idx], X_train_full.iloc[val_idx]
    y_train, y_val = y_train_full.iloc[train_idx], y_train_full.iloc[val_idx]

    model = xgb.XGBClassifier(eval_metric='mlogloss', objective='multi:softprob')
    model.fit(X_train.values, y_train.values)

    y_pred_val = model.predict(X_val.values)
    acc = accuracy_score(y_val, y_pred_val)
    f1 = f1_score(y_val, y_pred_val, average='weighted')
    print(f"Validation Accuracy: {acc:.4f} | F1-score: {f1:.4f}")
    fold_results.append({'fold': fold, 'accuracy': acc, 'f1': f1})

acc_mean = np.mean([r['accuracy'] for r in fold_results])
f1_mean = np.mean([r['f1'] for r in fold_results])
print(f"\nMédia Validation Accuracy: {acc_mean:.4f}")
print(f"Média Validation F1-score: {f1_mean:.4f}")

# ==============================
# MODELO FINAL COMPLETO
# ==============================

final_model = xgb.XGBClassifier(eval_metric='mlogloss', objective='multi:softprob')
final_model.fit(X_train_full.values, y_train_full.values)

feat_imp = pd.Series(final_model.feature_importances_, index=X_train_full.columns).sort_values(ascending=False)
feat_imp.to_csv('summary/Tess_light_all_feature_importance.csv')
print("\nTop 10 Feature importances:")
print(feat_imp.head(10))

# Avaliação no teste
y_pred_test = final_model.predict(X_test.values)
print("\n--- Base Model Test Results ---")
print(f"Accuracy: {accuracy_score(y_test, y_pred_test):.4f}")
print(f"F1-score: {f1_score(y_test, y_pred_test, average='weighted'):.4f}")

# Avaliação na blind validation
y_pred_blind = final_model.predict(X_blind.values)
print("\n--- Base Model Blind Results ---")
print(f"Accuracy: {accuracy_score(y_blind, y_pred_blind):.4f}")
print(f"F1-score: {f1_score(y_blind, y_pred_blind, average='weighted'):.4f}")

#joblib.dump(final_model, 'xgb_final_model.pkl')

# ==============================
# TESTES COM 90%, 95%, 99% DAS FEATURES
# ==============================

thresholds = [0.90, 0.95, 0.99]
results_summary = []

feat_imp_cumsum = feat_imp.cumsum()

for th in thresholds:
    selected_features = feat_imp_cumsum[feat_imp_cumsum <= th].index.tolist()
    print(th,selected_features)
    print(f"\n===== Testing with top {th*100:.0f}% feature importance =====")
    print(f"Selected features: {len(selected_features)}")

    X_train_sel = X_train_full[selected_features]
    X_test_sel = X_test[selected_features]
    X_blind_sel = X_blind[selected_features]

    model_sel = xgb.XGBClassifier(eval_metric='mlogloss', objective='multi:softprob')
    model_sel.fit(X_train_sel.values, y_train_full.values)

    # Test set
    y_pred_test_sel = model_sel.predict(X_test_sel.values)
    acc_test = accuracy_score(y_test, y_pred_test_sel)
    f1_test = f1_score(y_test, y_pred_test_sel, average='weighted')

    # Blind set
    y_pred_blind_sel = model_sel.predict(X_blind_sel.values)
    acc_blind = accuracy_score(y_blind, y_pred_blind_sel)
    f1_blind = f1_score(y_blind, y_pred_blind_sel, average='weighted')

    print(f"Test Accuracy: {acc_test:.4f} | F1: {f1_test:.4f}")
    print(f"Blind Accuracy: {acc_blind:.4f} | F1: {f1_blind:.4f}")
    report_test = classification_report(y_test, y_pred_test_sel)
    report_blind = classification_report(y_blind, y_pred_blind_sel)
    print(f"\n Number of Features for this model: {X_test_sel.shape[1]}")
    print(f"\n📊 Classification Test Report: TESS (TOI) Light Model with {th*100:.0f}% feature importance\n", report_test)
    print(f"\n📊 Classification Blind Report: TESS (TOI) Light Model with {th*100:.0f}% feature importance\n", report_blind)

    results_summary.append({
        'Threshold': f'{int(th*100)}%',
        'N_Features': len(selected_features),
        'Test_Accuracy': acc_test,
        'Test_F1': f1_test,
        'Blind_Accuracy': acc_blind,
        'Blind_F1': f1_blind
    })

    # Salvar modelo em formatos PKL e JSON
    #joblib.dump(model_sel, f'xgb_model_{int(th*100)}perc_features.pkl')
final_model.save_model(f'models/tess_lite.model')

# ==============================
# SUMÁRIO FINAL
# ==============================

results_df = pd.DataFrame(results_summary)
print("\n=== Summary of Reduced Feature Tests ===")
display(results_df)
results_df.to_csv("summary/Tess_light_feature_importance_summary.csv", index=False)

end = time()
print(f"\nTotal runtime: {end - start:.2f} seconds")


Train shape: (4592, 10)
Test shape: (1312, 10)
Blind validation shape: (656, 10)

Fold 1
Validation Accuracy: 0.7617 | F1-score: 0.7473

Fold 2
Validation Accuracy: 0.7769 | F1-score: 0.7645

Fold 3
Validation Accuracy: 0.7767 | F1-score: 0.7643

Fold 4
Validation Accuracy: 0.7680 | F1-score: 0.7567

Fold 5
Validation Accuracy: 0.7821 | F1-score: 0.7674

Média Validation Accuracy: 0.7731
Média Validation F1-score: 0.7600

Top 10 Feature importances:
st_disterr1       0.231626
st_tmag           0.111663
pl_insol          0.106755
st_dist           0.093417
pl_eqt            0.083912
pl_rade           0.083599
pl_tranmid        0.082880
st_disterr2       0.076727
pl_tranmiderr2    0.068754
st_raderr2        0.060667
dtype: float32

--- Base Model Test Results ---
Accuracy: 0.7759
F1-score: 0.7601

--- Base Model Blind Results ---
Accuracy: 0.7835
F1-score: 0.7657
0.9 ['st_disterr1', 'st_tmag', 'pl_insol', 'st_dist', 'pl_eqt', 'pl_rade', 'pl_tranmid', 'st_disterr2']

===== Testing with to

/opt/conda/lib/python3.10/site-packages/xgboost/sklearn.py:1028: UserWarning: [01:40:56] WARNING: /workspace/src/c_api/c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  self.get_booster().save_model(fname)


,Threshold,N_Features,Test_Accuracy,Test_F1,Blind_Accuracy,Blind_F1
0,90%,8,0.768293,0.751696,0.782012,0.765209
1,95%,9,0.766768,0.751540,0.768293,0.748367
2,99%,9,0.766768,0.751540,0.768293,0.748367



Total runtime: 4.97 seconds


In [65]:
import optuna
def objective(trial):
    params = {
        'objective': 'multi:softprob',
        'eval_metric': 'mlogloss',
        'num_class': len(classes),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 2.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 3.0),
        'gamma': trial.suggest_float('gamma', 0, 3.0),
        'n_estimators': trial.suggest_int('n_estimators', 300, 800)
    }
    model = xgb.XGBClassifier(**params)
    model.fit(X_train_full, y_train_full)
    preds = model.predict(X_test)
    return f1_score(y_test, preds, average='weighted')

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30)
print("Best params:", study.best_params)

[I 2025-10-05 13:25:55,397] A new study created in memory with name: no-name-5643c3cb-2f1b-45c3-8301-cbb7eb042d04
[I 2025-10-05 13:25:58,798] Trial 0 finished with value: 0.7704338154522922 and parameters: {'learning_rate': 0.024038403779744606, 'max_depth': 4, 'subsample': 0.9741691852657262, 'colsample_bytree': 0.6927561655886202, 'reg_alpha': 0.937554996896772, 'reg_lambda': 0.195065515844573, 'gamma': 1.4638722648687656, 'n_estimators': 432}. Best is trial 0 with value: 0.7704338154522922.
[I 2025-10-05 13:26:07,132] Trial 1 finished with value: 0.7726729848258529 and parameters: {'learning_rate': 0.08062597270281867, 'max_depth': 9, 'subsample': 0.7144118792860741, 'colsample_bytree': 0.6129306707804127, 'reg_alpha': 1.3699328795393788, 'reg_lambda': 0.8101409334080875, 'gamma': 1.4636614114412998, 'n_estimators': 633}. Best is trial 1 with value: 0.7726729848258529.
[I 2025-10-05 13:26:09,161] Trial 2 finished with value: 0.7786477503314863 and parameters: {'learning_rate': 0.241

Best params: {'learning_rate': 0.1008931455889113, 'max_depth': 6, 'subsample': 0.6289329866193453, 'colsample_bytree': 0.9589746181603868, 'reg_alpha': 1.1474238315799892, 'reg_lambda': 2.9991596011021215, 'gamma': 0.2367594379070153, 'n_estimators': 601}


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, classification_report, ConfusionMatrixDisplay
from sklearn.utils.class_weight import compute_class_weight
import xgboost as xgb
from time import time

# ================================
# 1️⃣ DATA SPLITTING
# ================================
start = time()

X = df_final.drop(['label'], axis=1)
y = df_final['label']

# Primeiro split: 70% treino, 30% restante
X_train_full, X_temp, y_train_full, y_temp = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=42
)

# Segundo split: 20% teste, 10% blind validation (dentro dos 30%)
X_test, X_blind, y_test, y_blind = train_test_split(
    X_temp, y_temp, test_size=0.3333, stratify=y_temp, random_state=42
)

print(f"Train shape: {X_train_full.shape}")
print(f"Test shape: {X_test.shape}")
print(f"Blind validation shape: {X_blind.shape}")

# ================================
# 2️⃣ CLASS WEIGHTS (for imbalance)
# ================================
from sklearn.utils.class_weight import compute_class_weight
classes = np.unique(y_train_full)
weights = compute_class_weight('balanced', classes=classes, y=y_train_full)
weights = weights / np.mean(weights)  # normalize around 1.0
class_weights = dict(zip(classes, weights))


# ================================
# 3️⃣ STRATIFIED K-FOLD TRAINING
# ================================
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

fold_results = []

for fold, (train_idx, val_idx) in enumerate(skf.split(X_train_full, y_train_full), 1):
    print(f"\nFold {fold}")
    X_train, X_val = X_train_full.iloc[train_idx], X_train_full.iloc[val_idx]
    y_train, y_val = y_train_full.iloc[train_idx], y_train_full.iloc[val_idx]

    # ================================
    # 4️⃣ XGBOOST MODEL
    # ================================
    model = xgb.XGBClassifier(
        objective='multi:softprob',
        eval_metric='mlogloss',
        learning_rate=0.05,   # back to your original
        max_depth=5,
        n_estimators=800,
        subsample=0.9,
        colsample_bytree=0.8,
        reg_lambda=1,
        reg_alpha=0,
        gamma=0,
        use_label_encoder=False
    )


    model.fit(
        X_train, y_train,
        sample_weight=y_train.map(class_weights),
        eval_set=[(X_val, y_val)],
        verbose=False
    )

    # ================================
    # 5️⃣ VALIDATION RESULTS
    # ================================
    y_pred_val = model.predict(X_val)
    acc = accuracy_score(y_val, y_pred_val)
    f1 = f1_score(y_val, y_pred_val, average='weighted')
    print(f"Validation Accuracy: {acc:.4f} | F1-score: {f1:.4f}")
    fold_results.append({'fold': fold, 'accuracy': acc, 'f1': f1})

# Resultados médios
acc_mean = np.mean([r['accuracy'] for r in fold_results])
f1_mean = np.mean([r['f1'] for r in fold_results])
print(f"\nMédia Validation Accuracy: {acc_mean:.4f}")
print(f"Média Validation F1-score: {f1_mean:.4f}")

# ================================
# 6️⃣ TRAIN FINAL MODEL ON FULL TRAIN SET
# ================================
final_model = xgb.XGBClassifier(
    objective='multi:softprob',
    eval_metric='mlogloss',
    learning_rate=0.05,   # back to your original
    max_depth=5,
    n_estimators=800,
    subsample=0.9,
    colsample_bytree=0.8,
    reg_lambda=1,
    reg_alpha=0,
    gamma=0,
    use_label_encoder=False
)

final_model.fit(
    X_train_full, y_train_full,
    sample_weight=y_train_full.map(class_weights),
    eval_set=[(X_test, y_test)],
    verbose=False
)

# ================================
# 7️⃣ FEATURE IMPORTANCE
# ================================
feat_imp = pd.Series(final_model.feature_importances_, index=X_train_full.columns).sort_values(ascending=False)
print("\nTop 15 feature importances:")
print(feat_imp.head(15))

# ================================
# 8️⃣ TEST EVALUATION
# ================================
y_pred_test = final_model.predict(X_test)
print("\nTest set results:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_test):.4f}")
print(f"F1-score: {f1_score(y_test, y_pred_test, average='weighted'):.4f}")
print(classification_report(y_test, y_pred_test))
ConfusionMatrixDisplay.from_estimator(final_model, X_test, y_test)

# ================================
# 9️⃣ BLIND VALIDATION
# ================================
y_pred_blind = final_model.predict(X_blind)
print("\nBlind validation results:")
print(f"Accuracy: {accuracy_score(y_blind, y_pred_blind):.4f}")
print(f"F1-score: {f1_score(y_blind, y_pred_blind, average='weighted'):.4f}")
print(classification_report(y_blind, y_pred_blind))
ConfusionMatrixDisplay.from_estimator(final_model, X_blind, y_blind)

print("\nTotal time:", round(time() - start, 2), "seconds")

# ===================================================
# 🔍 (OPTIONAL) OPTUNA HYPERPARAMETER TUNING BLOCK
# ===================================================
"""
import optuna
def objective(trial):
    params = {
        'objective': 'multi:softprob',
        'eval_metric': 'mlogloss',
        'num_class': len(classes),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 2.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 3.0),
        'gamma': trial.suggest_float('gamma', 0, 3.0),
        'n_estimators': trial.suggest_int('n_estimators', 300, 800)
    }
    model = xgb.XGBClassifier(**params, use_label_encoder=False)
    model.fit(X_train_full, y_train_full)
    preds = model.predict(X_test)
    return f1_score(y_test, preds, average='weighted')

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30)
print("Best params:", study.best_params)
"""


In [15]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.metrics import roc_auc_score, classification_report
import xgboost as xgb
from time import time

start = time()

# Features e label
X = df_final.drop(['label'], axis=1)
y = df_final['label']

# Split: 70% treino, 30% restante (teste + blind validation)
X_train_full, X_temp, y_train_full, y_temp = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=42
)

# Split: 20% teste, 10% blind validation
X_test, X_blind, y_test, y_blind = train_test_split(
    X_temp, y_temp, test_size=0.3333, stratify=y_temp, random_state=42
)

print(f"Train shape: {X_train_full.shape}")
print(f"Test shape: {X_test.shape}")
print(f"Blind validation shape: {X_blind.shape}")

# StratifiedKFold
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Modelo XGBoost
xgb_model = xgb.XGBClassifier(
    objective='multi:softprob',  # Multiclasse
    eval_metric='mlogloss'
)


# Grid de hiperparâmetros
param_grid = {
    'max_depth': [4, 5, 6],
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [100, 200, 300],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

# GridSearchCV com scoring para AUC multiclasse (One-vs-Rest)
grid = GridSearchCV(
    estimator=xgb_model,
    param_grid=param_grid,
    scoring='accuracy',  # AUC multiclasse
    cv=cv,
    verbose=3,
    n_jobs=-1
)

# Treinar
grid.fit(X_train_full, y_train_full)

print("\nMelhores hiperparâmetros encontrados:")
print(grid.best_params_)

# Melhor modelo
best_model = grid.best_estimator_

# Avaliação no teste
y_pred_test = best_model.predict(X_test)
y_proba_test = best_model.predict_proba(X_test)
auc_test = roc_auc_score(y_test, y_proba_test, multi_class='ovr', average='weighted')
print("\nTest set results:")
print(f"AUC (weighted, OVR): {auc_test:.4f}")
print(classification_report(y_test, y_pred_test))

# Avaliação blind validation
y_pred_blind = best_model.predict(X_blind)
y_proba_blind = best_model.predict_proba(X_blind)
auc_blind = roc_auc_score(y_blind, y_proba_blind, multi_class='ovr', average='weighted')
print("\nBlind validation results:")
print(f"AUC (weighted, OVR): {auc_blind:.4f}")
print(classification_report(y_blind, y_pred_blind))

# Feature importances
feat_imp = pd.Series(best_model.feature_importances_, index=X_train_full.columns).sort_values(ascending=False)
print("\nFeature importances:")
print(feat_imp)

end = time()
print("Time:", end-start)


Train shape: (4592, 10)
Test shape: (1312, 10)
Blind validation shape: (656, 10)
Fitting 5 folds for each of 108 candidates, totalling 540 fits
[CV 4/5] END colsample_bytree=0.8, learning_rate=0.01, max_depth=4, n_estimators=100, subsample=0.8;, score=0.766 total time=   0.7s
[CV 5/5] END colsample_bytree=0.8, learning_rate=0.01, max_depth=4, n_estimators=100, subsample=0.8;, score=0.780 total time=   0.7s
[CV 1/5] END colsample_bytree=0.8, learning_rate=0.01, max_depth=4, n_estimators=100, subsample=1.0;, score=0.740 total time=   0.7s
[CV 5/5] END colsample_bytree=0.8, learning_rate=0.01, max_depth=4, n_estimators=100, subsample=1.0;, score=0.777 total time=   0.7s
[CV 2/5] END colsample_bytree=0.8, learning_rate=0.01, max_depth=4, n_estimators=200, subsample=0.8;, score=0.771 total time=   1.4s
[CV 5/5] END colsample_bytree=0.8, learning_rate=0.01, max_depth=4, n_estimators=200, subsample=0.8;, score=0.782 total time=   1.4s
[CV 4/5] END colsample_bytree=0.8, learning_rate=0.01, max

In [171]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, classification_report
import xgboost as xgb

from time import time
start = time()
# Features e label
X = df_final.drop(['label'], axis=1)
y = df_final['label']

# Primeiro split: 70% treino, 30% restante
X_train_full, X_temp, y_train_full, y_temp = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=42
)

# Segundo split: 20% teste, 10% blind validation (dentro dos 30% restantes)
X_test, X_blind, y_test, y_blind = train_test_split(
    X_temp, y_temp, test_size=0.3333, stratify=y_temp, random_state=42
)

print(f"Train shape: {X_train_full.shape}")
print(f"Test shape: {X_test.shape}")
print(f"Blind validation shape: {X_blind.shape}")

# Stratified K-Fold dentro do conjunto de treino (70%)
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

fold_results = []

for fold, (train_idx, val_idx) in enumerate(skf.split(X_train_full, y_train_full), 1):
    print(f"\nFold {fold}")
    X_train, X_val = X_train_full.iloc[train_idx], X_train_full.iloc[val_idx]
    y_train, y_val = y_train_full.iloc[train_idx], y_train_full.iloc[val_idx]

    # Modelo XGBoost padrão
    model = xgb.XGBClassifier(eval_metric='mlogloss',objective='multi:softprob',colsample_bytree=1.0,
                              learning_rate=0.05, max_depth=4, n_estimators=300, subsample=0.8)
    model.fit(X_train, y_train)

    # Avaliação do fold
    y_pred_val = model.predict(X_val)
    acc = accuracy_score(y_val, y_pred_val)
    f1 = f1_score(y_val, y_pred_val, average='weighted')
    print(f"Validation Accuracy: {acc:.4f} | F1-score: {f1:.4f}")
    fold_results.append({'fold': fold, 'accuracy': acc, 'f1': f1})

# Resultados médios dos folds
acc_mean = np.mean([r['accuracy'] for r in fold_results])
f1_mean = np.mean([r['f1'] for r in fold_results])
print(f"\nMédia Validation Accuracy: {acc_mean:.4f}")
print(f"Média Validation F1-score: {f1_mean:.4f}")

# Treina modelo final no treino completo
final_model_tuned = xgb.XGBClassifier(eval_metric='mlogloss',objective='multi:softprob',colsample_bytree=1.0,
                              learning_rate=0.05, max_depth=4, n_estimators=300, subsample=0.8)
final_model_tuned.fit(X_train_full, y_train_full)

# Feature importances
feat_imp = pd.Series(final_model_tuned.feature_importances_, index=X_train_full.columns).sort_values(ascending=False)
print("\nFeature importances:")
print(feat_imp)

# Avaliação no teste
y_pred_test = final_model_tuned.predict(X_test)
print("\nTest set results:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_test):.4f}")
print(f"F1-score: {f1_score(y_test, y_pred_test, average='weighted'):.4f}")
print(classification_report(y_test, y_pred_test))

# Avaliação na blind validation
y_pred_blind = final_model_tuned.predict(X_blind)
print("\nBlind validation results:")
print(f"Accuracy: {accuracy_score(y_blind, y_pred_blind):.4f}")
print(f"F1-score: {f1_score(y_blind, y_pred_blind, average='weighted'):.4f}")
print(classification_report(y_blind, y_pred_blind))
end = time()
print("Time:", end-start)

Train shape: (4592, 10)
Test shape: (1312, 10)
Blind validation shape: (656, 10)

Fold 1
Validation Accuracy: 0.7802 | F1-score: 0.7664

Fold 2
Validation Accuracy: 0.7769 | F1-score: 0.7602

Fold 3
Validation Accuracy: 0.7930 | F1-score: 0.7776

Fold 4
Validation Accuracy: 0.7778 | F1-score: 0.7642

Fold 5
Validation Accuracy: 0.7952 | F1-score: 0.7797

Média Validation Accuracy: 0.7846
Média Validation F1-score: 0.7696

Feature importances:
st_disterr1       0.191193
pl_insol          0.119432
st_tmag           0.115148
st_dist           0.104338
pl_eqt            0.091221
pl_rade           0.083351
pl_tranmid        0.082476
st_disterr2       0.080957
pl_tranmiderr2    0.067293
pl_radeerr2       0.064591
dtype: float32

Test set results:
Accuracy: 0.7797
F1-score: 0.7612
              precision    recall  f1-score   support

           0       0.81      0.93      0.86       936
           1       0.64      0.39      0.48       137
           2       0.67      0.44      0.53       23

In [174]:
X_blind.to_csv('./models/tess/X_blind.csv')
y_blind.to_csv('./models/tess/y_blind.csv')
final_model_tuned.save_model("./models/tess/tess_model.json")
final_model_tuned.save_model("./models/tess/tess_model_tuned.json")

# Testar no Dataset todo

In [111]:
from sklearn.model_selection import cross_val_score
import numpy as np

# Ordena importâncias
feat_imp = pd.Series(final_model.feature_importances_, index=X_train.columns).sort_values(ascending=False)

results = {}
for k in [5, 10, 15, 20, 30, len(feat_imp)]:
    selected = feat_imp.index[:k]
    scores = cross_val_score(
        xgb.XGBClassifier(eval_metric='mlogloss'),
        X_train[selected],
        y_train,
        cv=5,
        scoring="accuracy"
    )
    results[k] = np.mean(scores)

print(results)

{5: 0.7245509648000891, 10: 0.7661928858736956, 15: 0.775993252886986, 20: 0.7860609093773749, 30: 0.7996715416411797, 52: 0.7955858310626703}
